## 张量的排序

张量的排序可以使用的
- tf.sort
- tf.argsor
- Topk
- Top-5 Acc
等方法


In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.3.0


### 排序

在排序前介绍一个洗牌和乱序的方法,沿着第一个轴进行乱序的。

- tf.random.shuffle()
>tf.random.shuffle(value, seed=None, name=None)  
Randomly shuffles a tensor along its first dimension.
- value: 需要洗牌的张量
- seed: 随机种子，保证数据的一致性，可以利用tf.random.set_seed来设置





In [8]:
a = tf.range(5)
print(a)
a_shuffle = tf.random.shuffle(a)
print(a_shuffle)
# 沿着第一个轴的方向进行展开
b = tf.constant([[1,2],[3,4],[5,6]])
b_sh = tf.random.shuffle((b))
print(b_sh)

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([4 1 0 2 3], shape=(5,), dtype=int32)
tf.Tensor(
[[3 4]
 [1 2]
 [5 6]], shape=(3, 2), dtype=int32)


sort 函数介绍：

- tf.sort (values, axis=- 1, direction='ASCENDING', name=None) 

>Args:
  - values: 1-D or higher numeric Tensor.
  - axis: The axis along which to sort. The default is -1, which sorts the last
    axis.
  - direction: The direction in which to sort the values ('ASCENDING' or
    'DESCENDING').
  - name: Optional name for the operation.


argsort： 对数据排序后索引的位置的记录

- tf.argsort: (values, axis=- 1, direction='ASCENDING', stable=False, name=None)

> Args:
 - values: 1-D or higher numeric Tensor.
- axis: The axis along which to sort. The default is -1, which sorts the last
    axis.
- direction: The direction in which to sort the values ('ASCENDING' or
    'DESCENDING').
- stable: If True, equal elements in the original tensor will not be
    re-ordered in the returned order. Unstable sort is not yet implemented, but will eventually be the default for performance reasons. If you require a stable order, pass stable=True for forwards compatibility.
- name: Optional name for the operation.





In [11]:
a = tf.random.shuffle(tf.range(5))
print(a)
# 降序排列
a_sort = tf.sort(a,direction='DESCENDING')
print(a_sort)
# 升序排序，默认
a_sort = tf.sort(a)
print(a_sort)

# 有一个有意的sort
a_argsort = tf.argsort(a)
print(a_argsort)

tf.Tensor([3 0 1 4 2], shape=(5,), dtype=int32)
tf.Tensor([4 3 2 1 0], shape=(5,), dtype=int32)
tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([1 2 4 0 3], shape=(5,), dtype=int32)


### tok_k 计算

在很多情况下我需要找到一个tensor中前几个最大值，就需要使用tok_k来获得。

tf.math.top_k: (input, k=1, sorted=True, name=None)

>Args:
  - input: 1-D or higher Tensor with last dimension at least k.
  - k: 0-D int32 Tensor. Number of top elements to look for along the last
    dimension (along each row for matrices).
  - sorted: If true the resulting k elements will be sorted by the values indescending order.
  - name: Optional name for the operation


返回值是一个 还有values indices的两个tensor:
- values： 找到的最大值数值
- indices: 表示找到前面几个最大值的对应的index



In [12]:
a =tf.random.shuffle(tf.range(5))
print(a)

res = tf.math.top_k(a,2)
print(res)
print(res.values)
print(res.indices)

tf.Tensor([2 4 0 3 1], shape=(5,), dtype=int32)
TopKV2(values=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 3])>, indices=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 3])>)
tf.Tensor([4 3], shape=(2,), dtype=int32)
tf.Tensor([1 3], shape=(2,), dtype=int32)


这里一个案例来实现计算分类问题的top_k精度的计算


In [48]:
import tensorflow as tf
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.random.set_seed(2021)

def accuracy(output,target,topk=(1,)):
    # 找到最大的k的值
    maxk = max(topk)
    # 获得本次传入的样本的个数
    batch_size = target.shape[0]
    # 获得top_k的indices的值
    pred = tf.math.top_k(output,maxk).indices
    print(pred)
    # 进行交换轴
    pred = tf.transpose(pred,perm=[1,0])
    # 将target的标签进行广播扩展，[10,b]
    target_ = tf.broadcast_to(target,pred.shape)
    # 进行比较获得最值的值
    corret = tf.equal(pred,target_)

    res = []
    # 开始对比需要比较的数据
    for k in topk:
        # 进行切片选取当前需要比较的数据
        corret_k = tf.cast(tf.reshape(corret[:k],[-1]),dtype=tf.float32)
        # 统计当前的正确数据的个数
        corret_k = tf.reduce_sum((corret_k))
        # 计算精度值
        acc = float(corret_k*(100.0/batch_size))
        res.append(acc)

    return res


In [49]:
# 随机的创建一个输出
output = tf.random.normal([10,6])
# print(output)
# 转换成概率
output = tf.math.softmax(output,axis=1)
# print(output)

# 创建一个标签，10个数，最大值为[1-5]
target = tf.random.uniform([10],maxval=6,dtype=tf.int32)
# 这个是模型输出的概率值
# print('prob:',output.numpy())
# 找到最大的值的位置
pred = tf.argmax(output,axis=1)

acc = accuracy(output,target,topk=(1,2,3,4,5,6))
print(acc)


pred = tf.cast(pred,tf.int32)
print(pred)
# 计算TOP_1的值
acc = tf.reduce_sum(tf.cast(tf.equal(pred,target),tf.float32))/target.shape[0]
print("acc: ",acc.numpy())



tf.Tensor(
[[0 4 2 1 5 3]
 [2 1 4 5 0 3]
 [1 0 5 4 3 2]
 [3 1 4 2 5 0]
 [4 0 5 1 2 3]
 [5 0 4 3 1 2]
 [4 1 5 0 3 2]
 [1 3 5 2 4 0]
 [4 3 2 1 0 5]
 [4 5 0 1 3 2]], shape=(10, 6), dtype=int32)
[20.0, 40.0, 60.0, 90.0, 100.0, 100.0]
tf.Tensor([0 2 1 3 4 5 4 1 4 4], shape=(10,), dtype=int32)
acc:  0.2
